In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import sys
dir="/content/drive/My Drive/Hybrid Model/CSEF"
sys.path.append(dir)
print(dir)
from processing import ADOS_Data

/content/drive/My Drive/Hybrid Model/CSEF


In [ ]:
from cross_validation import CVP_Set
from sklearn.linear_model import LinearRegression, LogisticRegression, Ridge
from sklearn.svm import LinearSVC, SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from classifiers import RegClassifier, BinClassifier
from sklearn.metrics import roc_auc_score, average_precision_score, classification_report, confusion_matrix, precision_recall_curve, auc

import matplotlib.pyplot as plt
import pandas as pd

# def test_func(pred_score, predlabels, name='lr5'):
#     y_true = [int(v!=0) for v in list(predlabels)]
#     print('Confusion matrix:')
#     cm = confusion_matrix([1-x for x in y_true], [1-int(x > 0.5) for x in pred_score])
#     print(cm)
#     print('Precision: {}'.format(float(cm[0][0])/(cm[0][0]+cm[1][0])))
#     print('Recall/Sensitivity: {}'.format(float(cm[0][0])/(cm[0][0]+cm[0][1])))
#     print('Specificity: {}'.format(float(cm[1][1])/(cm[1][1]+cm[1][0])))
#     print ('class report')
#     print (classification_report([1-x for x in y_true], [1-int(x > 0.5) for x in pred_score]))
def test_func(pred_score, predlabels, name = 'lr5'):
  y_true = [int(v!=0) for v in list(predlabels)]
    print('Confusion matrix:')
    cm = confusion_matrix([1-x for x in y_true], [1-int(x > 0.5) for x in pred_score])


################### data loading ###############
MODULE = 'm2'
FEATURE_SET = ['A3', 'A5', 'B1', 'B2', 'B10']
N_FOLD = 10
PRED_RATIO = 0.2
SCALING_PARAM = 4
# Binary or Replacement
MISSING_VALUE_STRATEGY = 'Binary'
# poly, linear, indicator, interaction_ind, pca_comp
PROCESSING_STRATEGY = 'linear'

POLY_DEGREE = 2
NORMALIZE = True

ADOS_FILE = ADOS_FILE = dir + ("/ados_m2_allData.csv")
label_id = "ASD"
label_age = "age_months"
label_gender = "male"
columns_to_delete = ["Subject.Id", "Diagnosis"]
sub_diagnosis_id = ["social_affect_calc","restricted_repetitive_calc","SA_RRI_total_calc","severity_calc"]

# We import the data
data = ADOS_Data.read_csv(ADOS_FILE)
sub_diagnosis = data[sub_diagnosis_id]

# We drop the columns that are not interesting for us, and the row with no label
data.select_good_columns(columns_to_delete+sub_diagnosis_id)

print('gendering')
print((data[data['ASD'] == 1][['male']]).sum())


data.full_preprocessing(NORMALIZE, MISSING_VALUE_STRATEGY, PROCESSING_STRATEGY, [label_age], label_gender, label_id)

'''
we are only training on the 5 features above (doing the "select good columns")
['A3', 'A5', 'B1', 'B2', 'B10'], very likely that's why the training runs very fast
'''

print(data.shape)
if FEATURE_SET != []:
    data.select_good_columns(FEATURE_SET, keep_the_column=True)
    data[:]=data[FEATURE_SET]
print("HERE")
print(data[:])
cv_set = CVP_Set(data, data.labels, N_FOLD, PRED_RATIO)

'''
this is the binary classifier the author use. this is calling the LogisticRegression from sklearn, but then make it discrete.
i.e. from any number in 0 ~ 1(0.01, 0.8...) -> only category(categories) we are looking for, in this case just 0 and 1
'''
lr = BinClassifier(proc=LogisticRegression(C=0.045, penalty='l2',class_weight='balanced'), severity=True)
lr.fit(cv_set.cv_feat, cv_set.cv_labels)




''' this file actually located in the same diretory as the RUNME_L5.ipynb. Please lemme know if you still got failure on this run '''
FILENAME = dir + ("/primary_dataset.csv")
columns_to_delete = ['child_id','scorer_id','video_file']
pred_feat = ADOS_Data.read_csv(FILENAME,skip_blank_lines=True)
# pred_feat = pred_feat[0:19]
pred_feat = ADOS_Data(pred_feat)
ytrue = pred_feat["ASD"]
# We drop the columns that are not interesting for us, and the row with no label
pred_feat.select_good_columns(columns_to_delete)
print("HI?!")
print(pred_feat)




pred_feat.full_preprocessing(NORMALIZE, MISSING_VALUE_STRATEGY, PROCESSING_STRATEGY, [label_age], label_gender, label_id)
if FEATURE_SET != []:
    pred_feat.select_good_columns(FEATURE_SET, keep_the_column=True)
    # pred_feat[:]=pred_feat[FEATURE_SET]
print("=====starting here is the result on this model======")
#print(pred_feat)
test_func(list(lr.predict(pred_feat)), ytrue)
print(list(lr.predict(pred_feat)))

svc = BinClassifier(proc = SVC(C = 1), severity = False)
svc.fit(cv_set.cv_feat, cv_set.cv_labels)
svc_dec_vals = svc.decision_function(pred_feat)

''' optional of printing this to see prediction details '''
'''
print("---Printing probabilities of each input patient's chance of ASD---")
print("you can always just get this result, and simply classify yes/no ADS comparing the probability with 0.5")
LogisticProbabilities = (lr.predict_proba(pred_feat))
for line in LogisticProbabilities:
    print(line)
print("---End of Printing Probabilities---")
print("total number of result here is the same as the testing file : ", len(LogisticProbabilities))


l = list(zip(LogisticProbabilities, list(lr.predict(pred_feat))))
with open('results_lr_5.csv', 'w') as f:
    for i in range(len(l)):
        f.write(str(l[i])+'\n') '''
print('svc')
test_func(svc_dec_vals, ytrue, 'SVC')
print(svc_dec_vals)
#weighted average
# result1 = svc_dec_vals
# minval = result1.min()
# maxval = result1.max()
# result1 = (result1-minval)/(maxval-minval)
# print(result1)

hello!
      A1   A2  A3  A4  A5  A6  A7  ...  D4  E1  E2  E3  age_months  male  ASD
0      0  1.0   2   0   1   2   0  ...   3   0   1   2        65.0     1    2
1      1  0.0   1   2   1   1   0  ...   1   1   0   1        36.0     0    2
2      1  2.0   2   1   2   2   2  ...   2   1   2   2        58.0     0    2
3      0  1.0   1   0   1   2   0  ...   1   0   1   1        49.0     1    2
4      1  0.0   1   2   2   2   0  ...   1   1   0   1        49.0     0    2
...   ..  ...  ..  ..  ..  ..  ..  ...  ..  ..  ..  ..         ...   ...  ...
1384   0  0.0   2   2   2   2   0  ...   2   0   0   0       107.0     0    2
1385   1  2.0   2   2   1   2   1  ...   2   0   0   0        79.0     1    2
1386   0  1.0   2   2   2   2   1  ...   2   1   0   0        60.0     0    2
1387   0  1.0   2   1   2   1   0  ...   2   1   0   0        81.0     1    2
1388   0  1.0   1   1   0   1   0  ...   1   0   0   0        62.0     1    1

[1389 rows x 31 columns]
gendering
male    163
dtype: in

/content/drive/My Drive/Hybrid Model/CSEF/processing.py:76: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  self.labels = self[label_id]


      A3  A5  B1  B2  B10
0      2   1   2   1    1
1      1   1   2   1    0
2      2   2   2   2    2
3      1   1   2   2    0
4      1   2   2   1    0
...   ..  ..  ..  ..  ...
1384   2   2   2   1    1
1385   2   1   2   2    2
1386   2   2   2   1    2
1387   2   2   2   1    2
1388   1   0   2   1    1

[1389 rows x 5 columns]
hello!
         updatedAt  question_set  A3  ...  male      createdAt    updatedAt.1
0    6/26/17 16:43             1   8  ...     1  6/26/17 16:43  6/26/17 16:43
1    6/26/17 16:57             1   0  ...     1  6/26/17 16:57  6/26/17 16:57
2    6/26/17 17:29             1   0  ...     0  6/26/17 17:29  6/26/17 17:29
3    6/26/17 17:35             1   2  ...     1  6/26/17 17:35  6/26/17 17:35
4    6/26/17 18:36             1   0  ...     1  6/26/17 18:36  6/26/17 18:36
..             ...           ...  ..  ...   ...            ...            ...
462  11/5/17 17:35             1   0  ...     0  11/5/17 17:35  11/5/17 17:35
463  11/5/17 17:45             1

/content/drive/My Drive/Hybrid Model/CSEF/processing.py:76: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  self.labels = self[label_id]


                    updatedAt  question_set    A3    A5   B10  question3  \
0.6375                    NaN           NaN   NaN   NaN   NaN        NaN   
0.75                      NaN           NaN   NaN   NaN   NaN        NaN   
0.8625                    NaN           NaN   NaN   NaN   NaN        NaN   
0.9749999999999999        NaN           NaN   NaN   NaN   NaN        NaN   
1                         NaN         467.0  45.0  87.0  87.0      117.0   
...                       ...           ...   ...   ...   ...        ...   
7/6/17 21:42              1.0           NaN   NaN   NaN   NaN        NaN   
7/6/17 21:46              1.0           NaN   NaN   NaN   NaN        NaN   
7/7/17 18:32              1.0           NaN   NaN   NaN   NaN        NaN   
7/7/17 19:40              1.0           NaN   NaN   NaN   NaN        NaN   
7/7/17 19:47              1.0           NaN   NaN   NaN   NaN        NaN   

                    question4  question5  question6  question7    B1  \
0.6375         